In [10]:
import os
import openai

from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")

In [12]:

SANDBOX_DIR = "/Users/aidand/dev/babyagi/sandbox"
def create_file(path, content):
    with open(os.path.join(SANDBOX_DIR, path), "w") as f:
        f.write(content)
    print(f"Created file {os.path.join(SANDBOX_DIR, path)}")

# Does the same thing as create_file - but nice to have a separate function for updating files
# So the LLM has the option to update files if it wants to - if that makes more sense than creating a new file
def update_file(path, content):
    with open(os.path.join(SANDBOX_DIR, path), "w") as f:
        f.write(content)
    print(f"Updated file {os.path.join(SANDBOX_DIR, path)}")

def delete_file(path):
    os.remove(os.path.join(SANDBOX_DIR, path))
    print(f"Deleted file {os.path.join(SANDBOX_DIR, path)}")


TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "create_file",
            "description": "Create a file with the given name and content",
            "parameters": {
                "type": "object",
                "properties": {
                    "path": {
                        "type": "string",
                        "description": "The relative path to the file to create",
                    },
                    "content": {
                        "type": "string",
                        "description": "The content of the file to create",
                    },
                },
                "required": ["path", "content"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "update_file",
            "description": "Update a file with the given name and content",
        },
        "parameters": {
            "type": "object",
            "properties": {
                "path": {
                    "type": "string",
                    "description": "The relative path to the file to update",
                },
                "content": {
                    "type": "string",
                    "description": "The content of the file to update",
                },
            },
            "required": ["path", "content"],
        },
    },
    {
        "type": "function",
        "function": {
            "name": "delete_file",
            "description": "Update a file with the given name and content",
        },
        "parameters": {
            "type": "object",
            "properties": {
                "path": {
                    "type": "string",
                    "description": "The relative path to the file to delete",
                },
            },
            "required": ["path"],
        },
    },
]

In [20]:
from openai import OpenAI
from pydantic import BaseModel
import json

PROGRAM_OBJECTIVE="download a PDF from the internet and convert it to a text file."

CODER_AGENT_SYSTEM_PROMPT="""
You are a software engineer who is writing code to build a small python codebase that will {PROGRAM_OBJECTIVE}.
"""

task_list = []

from typing import List

class Plan(BaseModel):
    steps: List[str]

client = OpenAI(api_key=OPENAI_API_KEY)

# Coding agent creates a step by step plan
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": CODER_AGENT_SYSTEM_PROMPT},
        {"role": "user", "content": """
         Create a step by step plan to complete the task of creating a codebase that will {PROGRAM_OBJECTIVE}.
         You have 3 different operations you can perform. create_file(path, content), update_file(path, content), delete_file(path).
         Limit your steps to only using these operations.
         Please include a README.md file in the root of the codebase that describes the codebase and how to run it.
         Please include a requirements.txt file in the root of the codebase that describes the dependencies of the codebase.
         """},
    ],
    tools=TOOLS,
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "Plan",
            "description": "A plan to complete the task of creating a codebase that will {PROGRAM_OBJECTIVE}.",
            "strict": True,
            "schema": {
                "type": "object",
                "properties": {
                    "steps": {"type": "array", "items": {"type": "string"}},
                },
                "required": ["steps"],
                "additionalProperties": False,
            },
        },
    },
)
plan = json.loads(response.choices[0].message.content)
print("Coder Agent - Plan")
for step in plan["steps"]:
    print(step)

# Coding agent prioritizes the plan


# Coding agent executes the plan

# Review agent reviews the code
# Review agent prioritizes the comments


Coder Agent - Plan
Create a README.md file describing the codebase and how to run it.
Create a requirements.txt file listing the dependencies for the codebase.
Create the main Python script file for the codebase.
Implement the core functionality of the codebase in the main script file.
Update the README.md file with usage instructions for the main script.
Test the codebase to ensure everything works as expected.
